In [1]:
import csv
import json
import numpy as np
import pandas as pd
import re
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer 
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
import string
import pickle
import collections
import matplotlib.pyplot as plt
import ast
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import confusion_matrix

lmtz = WordNetLemmatizer()

[nltk_data] Downloading package punkt to /Users/boomssv/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/boomssv/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/boomssv/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /Users/boomssv/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
import os
os.getcwd()

'/Users/boomssv/Documents/GitHub/stat628-module3-group4-fall2022-uwmadison/code/Review Analysis'

### 1. Import review_with_rating from EDA part

In [134]:
bars_review = pd.read_csv("../../data/bars_list_ca.csv").drop(columns=['Unnamed: 0'])
bars_review = pd.DataFrame(data = bars_review, columns = ["business_id","stars_x","text"])
bars_review = bars_review.sort_values(by = ["business_id"]).reset_index().drop(columns = ["index"])

In [135]:
bars_review

,business_id,stars_x,text
0,-BdYhP-12elmFV7oB1iv4A,5.0,"Great beer selection, flat breads are great, g..."
1,-BdYhP-12elmFV7oB1iv4A,5.0,Great new addition to the Mesa. Great selecti...
2,-BdYhP-12elmFV7oB1iv4A,5.0,Corner Tap has been nothing but the friendlies...
3,-BdYhP-12elmFV7oB1iv4A,5.0,Awesome spot and much needed on the Mesa! Lov...
4,-BdYhP-12elmFV7oB1iv4A,5.0,This is the place to go and decompress. The pe...
...,...,...,...
44688,zkMqKD6IZPcU10g0a2APmg,5.0,First time to Santa Barbara and this shop but ...
44689,zkMqKD6IZPcU10g0a2APmg,5.0,La Aroma De Havana is a great cigar lounge I w...
44690,zkMqKD6IZPcU10g0a2APmg,5.0,Cigar and beer lover? Then it's a must stop wh...
44691,zkMqKD6IZPcU10g0a2APmg,5.0,I would vacation in Santa Barbara just to smok...


In [59]:
review_with_rating = pd.read_csv("../../data/review_with_rating.csv")

In [60]:
review_with_rating

,rating,review_words
0,5.0,"['bit', 'weary', 'trying', 'shellfish', 'compa..."
1,5.0,"['great', 'beautiful', 'spot', 'wine', 'tastin..."
2,3.0,"['tough', 'one', 'merit', 'wine', 'fairly', 'a..."
3,5.0,"['love', 'trying', 'fresh', 'seafood', 'pier',..."
4,5.0,"['super', 'delish', 'frill', 'great', 'sea', '..."
...,...,...
44688,5.0,"['favorite', 'brunch', 'spot', 'santa', 'barba..."
44689,5.0,"['combo', 'meat', 'plate', 'good', 'ton', 'foo..."
44690,2.0,"['table', 'got', 'many', 'weird', 'look', 'emp..."
44691,5.0,"['could', 'give', '5', 'star', 'would', 'reser..."


### 2. Get the adjectives from review_words

In [61]:
nltk.pos_tag(["tough","got","star"])

[('tough', 'JJ'), ('got', 'VBD'), ('star', 'NN')]

In [62]:
adjectives = []
for row in range(len(review_with_rating)):
    words = ast.literal_eval(review_with_rating["review_words"][row])
    # put category for each word
    tag = nltk.pos_tag(words)
    # select only adjectives
    adj = [word for word, tag in tag if tag == "JJ"]
    adjectives.append(adj)

In [63]:
adjectives

[['weary',
  'shellfish',
  'many',
  'monterey',
  'feast',
  'captive',
  'high',
  'perennial',
  'good',
  'initial',
  'small',
  'plan',
  'peak',
  'special',
  'local',
  'awesome',
  'great',
  'hearty',
  'good',
  'much',
  'shrimp',
  'bisque',
  'prattling',
  'glorious',
  'white',
  'thick',
  'separate',
  'good',
  'great',
  'satisfied',
  'special',
  'overall',
  'great',
  'favorite',
  'safe'],
 ['great', 'beautiful', 'wine', 'tried', 'great', 'good'],
 ['tough',
  'average',
  'understand',
  'lable',
  'available',
  'wonderful',
  'nicer',
  'enjoy',
  'super',
  'expensive',
  'free',
  'wine',
  'winery',
  'give',
  'complimentary',
  'carr',
  'new',
  'wine',
  'limited',
  'beautiful',
  'avoid',
  'much',
  'wine',
  'urban'],
 ['fresh',
  'wharf',
  'fresh',
  'gorgeous',
  'top',
  'beautiful',
  'wrong',
  'good',
  'lunchearly',
  'table',
  'outdoor',
  'right',
  'sunscreen',
  'local',
  'salad',
  'steal',
  'huge',
  'delicious',
  'fresh',
  's

### 3. Create dictionary counting adjectives from reviews

In [64]:
# Collect all adjective
all_adj = [adj for i in adjectives for adj in i]
adj_freq = dict(collections.Counter(all_adj))

In [65]:
adj_freq

{'weary': 18,
 'shellfish': 403,
 'many': 2894,
 'monterey': 6,
 'feast': 10,
 'captive': 3,
 'high': 1662,
 'perennial': 3,
 'good': 23313,
 'initial': 86,
 'small': 3718,
 'plan': 26,
 'peak': 45,
 'special': 3438,
 'local': 3279,
 'awesome': 2563,
 'great': 25744,
 'hearty': 31,
 'much': 3311,
 'shrimp': 774,
 'bisque': 26,
 'prattling': 1,
 'glorious': 30,
 'white': 1475,
 'thick': 265,
 'separate': 176,
 'satisfied': 178,
 'overall': 2519,
 'favorite': 3598,
 'safe': 242,
 'beautiful': 2286,
 'wine': 3433,
 'tried': 604,
 'tough': 325,
 'average': 571,
 'understand': 152,
 'lable': 1,
 'available': 826,
 'wonderful': 2756,
 'nicer': 52,
 'enjoy': 396,
 'super': 1966,
 'expensive': 933,
 'free': 1386,
 'winery': 95,
 'give': 1126,
 'complimentary': 442,
 'carr': 23,
 'new': 3196,
 'limited': 234,
 'avoid': 106,
 'urban': 111,
 'fresh': 6068,
 'wharf': 122,
 'gorgeous': 415,
 'top': 2239,
 'wrong': 1125,
 'lunchearly': 5,
 'table': 5896,
 'outdoor': 1084,
 'right': 2246,
 'sunscreen

In [66]:
len(adj_freq)

18020

### 4. Create matrix of frequency

In [67]:
corpus = pd.DataFrame(data = list(bars_review["text"]), columns = ["text"])
vectorizer = CountVectorizer(vocabulary = list(adj_freq.keys()), min_df = 0)
freq_matrix = pd.DataFrame(data = vectorizer.fit_transform(corpus["text"].values).toarray(), columns = vectorizer.get_feature_names())

In [68]:
corpus

,text
0,"Great beer selection, flat breads are great, g..."
1,Great new addition to the Mesa. Great selecti...
2,Corner Tap has been nothing but the friendlies...
3,Awesome spot and much needed on the Mesa! Lov...
4,This is the place to go and decompress. The pe...
...,...
44688,First time to Santa Barbara and this shop but ...
44689,La Aroma De Havana is a great cigar lounge I w...
44690,Cigar and beer lover? Then it's a must stop wh...
44691,I would vacation in Santa Barbara just to smok...


In [69]:
freq_matrix

,weary,shellfish,many,monterey,feast,captive,high,perennial,good,initial,...,not_fear,focusing,cassava,privileged,tupperware,biodegradable,trendyromantic,delicious30,agh,hasslefree
0,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44688,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
44689,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
44690,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
44691,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 5. Fit the model using Naive Bayes classifier

In [70]:
# First we classify stars to positive/negative (4 stars or more are positive)
bars_review["stars_x"] = np.where(bars_review["stars_x"] >= 4, "positive", "negative")

In [71]:
bars_review

,business_id,stars_x,text
38648,-BdYhP-12elmFV7oB1iv4A,positive,"Great beer selection, flat breads are great, g..."
37534,-BdYhP-12elmFV7oB1iv4A,positive,Great new addition to the Mesa. Great selecti...
37098,-BdYhP-12elmFV7oB1iv4A,positive,Corner Tap has been nothing but the friendlies...
37831,-BdYhP-12elmFV7oB1iv4A,positive,Awesome spot and much needed on the Mesa! Lov...
38402,-BdYhP-12elmFV7oB1iv4A,positive,This is the place to go and decompress. The pe...
...,...,...,...
28167,zkMqKD6IZPcU10g0a2APmg,positive,First time to Santa Barbara and this shop but ...
28571,zkMqKD6IZPcU10g0a2APmg,positive,La Aroma De Havana is a great cigar lounge I w...
28194,zkMqKD6IZPcU10g0a2APmg,positive,Cigar and beer lover? Then it's a must stop wh...
28204,zkMqKD6IZPcU10g0a2APmg,positive,I would vacation in Santa Barbara just to smok...


In [72]:
model = MultinomialNB()
model.fit(freq_matrix, bars_review['stars_x'])

MultinomialNB()

In [73]:
# Test the prediction performance

def prediction(adj):
    return model.predict(vectorizer.transform([adj]))[0]

In [74]:
prediction("worst")

'negative'

In [75]:
adj_classifier = {}
for adj in adj_freq.keys():
    adj_classifier[adj] = prediction(adj)

In [76]:
adj_classifier

{'weary': 'positive',
 'shellfish': 'positive',
 'many': 'positive',
 'monterey': 'positive',
 'feast': 'positive',
 'captive': 'negative',
 'high': 'positive',
 'perennial': 'positive',
 'good': 'positive',
 'initial': 'positive',
 'small': 'positive',
 'plan': 'positive',
 'peak': 'positive',
 'special': 'positive',
 'local': 'positive',
 'awesome': 'positive',
 'great': 'positive',
 'hearty': 'positive',
 'much': 'positive',
 'shrimp': 'positive',
 'bisque': 'positive',
 'prattling': 'positive',
 'glorious': 'positive',
 'white': 'positive',
 'thick': 'positive',
 'separate': 'positive',
 'satisfied': 'positive',
 'overall': 'positive',
 'favorite': 'positive',
 'safe': 'positive',
 'beautiful': 'positive',
 'wine': 'positive',
 'tried': 'positive',
 'tough': 'positive',
 'average': 'negative',
 'understand': 'negative',
 'lable': 'negative',
 'available': 'positive',
 'wonderful': 'positive',
 'nicer': 'positive',
 'enjoy': 'positive',
 'super': 'positive',
 'expensive': 'positive'

### 6. Get the nouns from review_words and count

In [77]:
nouns = []
for row in range(len(review_with_rating)):
    words = ast.literal_eval(review_with_rating["review_words"][row])
    # put category for each word
    tag = nltk.pos_tag(words)
    # select only adjectives
    noun = [word for word, tag in tag if tag == "NN"]
    nouns.append(noun)

In [78]:
# Collect all nouns
all_noun = [noun for i in nouns for noun in i]
noun_freq = dict(collections.Counter(all_noun))

In [79]:
noun_freq

{'bit': 3072,
 'company': 510,
 'see': 732,
 'cannery': 2,
 'audience': 15,
 'substandard': 8,
 'fare': 169,
 'price': 4367,
 'review': 2833,
 'yelp': 1136,
 'observation': 12,
 'place': 23142,
 'visit': 2399,
 'hour': 4584,
 'minute': 3155,
 'dinner': 6079,
 'rush': 176,
 'rock': 971,
 'crab': 3401,
 'lb': 66,
 'california': 562,
 'salad': 3758,
 'choice': 1670,
 'soupchowder': 1,
 'look': 1206,
 'tray': 85,
 'wife': 2135,
 'saladchowder': 1,
 'combo': 339,
 'portion': 2394,
 'start': 814,
 'chowder': 3653,
 'side': 2957,
 'dish': 2597,
 'juicy': 507,
 'buttery': 252,
 'chunk': 285,
 'meat': 1928,
 'crack': 91,
 'shell': 456,
 'splinter': 2,
 'sign': 391,
 'perfection': 771,
 'amount': 1077,
 'work': 1373,
 'youre': 2353,
 'meal': 4381,
 'value': 367,
 'sbsc': 13,
 'trip': 1331,
 'proposition': 6,
 'spot': 4156,
 'wine': 9655,
 'merit': 7,
 'winery': 809,
 'label': 37,
 'premium': 42,
 'week': 1266,
 'location': 2984,
 'outdoor': 846,
 'patio': 2367,
 'staff': 7044,
 'super': 1006,
 '

In [80]:
collections.Counter(all_noun).most_common(100)

[('food', 29525),
 ('place', 23142),
 ('service', 18685),
 ('time', 14450),
 ('restaurant', 10491),
 ('wine', 9655),
 ('barbara', 7913),
 ('bar', 7521),
 ('staff', 7044),
 ('order', 6695),
 ('santa', 6694),
 ('menu', 6650),
 ('experience', 6497),
 ('dinner', 6079),
 ('night', 6065),
 ('beer', 5509),
 ('drink', 5334),
 ('everything', 4932),
 ('day', 4918),
 ('hour', 4584),
 ('meal', 4381),
 ('price', 4367),
 ('way', 4352),
 ('friend', 4337),
 ('star', 4298),
 ('thing', 4180),
 ('spot', 4156),
 ('area', 4069),
 ('burger', 4016),
 ('flavor', 4005),
 ('seafood', 3842),
 ('server', 3808),
 ('salad', 3758),
 ('clam', 3659),
 ('chowder', 3653),
 ('cocktail', 3445),
 ('lot', 3410),
 ('crab', 3401),
 ('table', 3288),
 ('try', 3205),
 ('minute', 3155),
 ('bit', 3072),
 ('taste', 3061),
 ('view', 3014),
 ('sauce', 3008),
 ('wait', 2997),
 ('get', 2997),
 ('location', 2984),
 ('side', 2957),
 ('plate', 2895),
 ('reservation', 2888),
 ('chicken', 2886),
 ('selection', 2835),
 ('review', 2833),
 ('l

#### We still need to pick the words that are useful for the analysis

In [81]:
review_with_rating

,rating,review_words
0,5.0,"['bit', 'weary', 'trying', 'shellfish', 'compa..."
1,5.0,"['great', 'beautiful', 'spot', 'wine', 'tastin..."
2,3.0,"['tough', 'one', 'merit', 'wine', 'fairly', 'a..."
3,5.0,"['love', 'trying', 'fresh', 'seafood', 'pier',..."
4,5.0,"['super', 'delish', 'frill', 'great', 'sea', '..."
...,...,...
44688,5.0,"['favorite', 'brunch', 'spot', 'santa', 'barba..."
44689,5.0,"['combo', 'meat', 'plate', 'good', 'ton', 'foo..."
44690,2.0,"['table', 'got', 'many', 'weird', 'look', 'emp..."
44691,5.0,"['could', 'give', '5', 'star', 'would', 'reser..."


#### Merge the data having 1 business id for each row (combine reviews)

In [82]:
business = pd.read_csv("../../data/bars_list.csv").drop(columns = ["Unnamed: 0"])

In [270]:
business_CA = pd.DataFrame(business[business["state"] == "CA"])
business_CA = business_CA.sort_values(by = ["business_id"]).reset_index().drop(columns = ["index"])

In [136]:
review_i = []
bus_id = []

for row in range(len(bars_review)):
    if bars_review["business_id"][row] not in bus_id:
        review_i.append(row)
        bus_id.append(bars_review["business_id"][row])

In [186]:
len(review_i)

163

In [204]:
bars_review[44619:]

,business_id,stars_x,text
44619,zkMqKD6IZPcU10g0a2APmg,5.0,Great atmosphere and Moses is very hospitable ...
44620,zkMqKD6IZPcU10g0a2APmg,5.0,I love their selection of cigars. I've interac...
44621,zkMqKD6IZPcU10g0a2APmg,5.0,Spent this past Saturday here for a birthday p...
44622,zkMqKD6IZPcU10g0a2APmg,5.0,Best cigar lounge in America hands down!! Supe...
44623,zkMqKD6IZPcU10g0a2APmg,4.0,This tobacco shop is a great jewel in the hear...
...,...,...,...
44688,zkMqKD6IZPcU10g0a2APmg,5.0,First time to Santa Barbara and this shop but ...
44689,zkMqKD6IZPcU10g0a2APmg,5.0,La Aroma De Havana is a great cigar lounge I w...
44690,zkMqKD6IZPcU10g0a2APmg,5.0,Cigar and beer lover? Then it's a must stop wh...
44691,zkMqKD6IZPcU10g0a2APmg,5.0,I would vacation in Santa Barbara just to smok...


In [198]:
combined_review = []
for i in range(len(review_i)):
    print(i)
    print(review_i[i])
    if i == len(review_i)-1:
        combined_review.append(list(bars_review["text"][review_i[i]:]))
    else:
        combined_review.append(list(bars_review["text"][review_i[i]:review_i[i+1]]))

0
0
1
44
2
166
3
409
4
1080
5
1520
6
1763
7
1974
8
2884
9
3055
10
3363
11
3509
12
3741
13
5275
14
5542
15
5619
16
5705
17
5752
18
6103
19
6155
20
6391
21
6430
22
6816
23
7100
24
7522
25
7705
26
7862
27
8874
28
9053
29
9060
30
9098
31
9354
32
9374
33
9486
34
10142
35
10182
36
10198
37
10327
38
11405
39
11947
40
12101
41
12194
42
12654
43
12678
44
13054
45
13513
46
13609
47
13657
48
13691
49
13854
50
13897
51
14831
52
14944
53
14987
54
15017
55
15329
56
15821
57
15845
58
16125
59
16194
60
16831
61
16866
62
17010
63
17222
64
17435
65
17902
66
17979
67
18308
68
18337
69
18502
70
20946
71
21082
72
21156
73
22283
74
25286
75
25493
76
25533
77
26026
78
26038
79
26060
80
26114
81
26123
82
26347
83
26781
84
27209
85
27464
86
27504
87
27522
88
27630
89
27992
90
28230
91
28787
92
28879
93
28935
94
28940
95
29215
96
29290
97
29324
98
29379
99
29589
100
30069
101
30475
102
30560
103
30662
104
30678
105
31006
106
31089
107
31107
108
31141
109
31248
110
31525
111
31552
112
32307
113
32473
114
32482
1

In [255]:
combined_review[1]

["My husband and I stopped here once while we were in town last month. What really stood out to me was the comfortable, yet chic environment. They have a lot of very delicious sparkling wines- which you don't see a lot of in this area!\nWe will definitely be back!",
 "Girl friends and I made an unplanned stop at Riverbench and were blown away by the amazing time we had during our tasting, especially their sparkling brut rosé, I had to write a review for our pourer Jordie, who took very good care of us and was extremely passionate and knowledgeable about their products. She took the time to explain in detail each wine we chose for our flights. We also had 3 small dogs with us and she came out with dog treats to make sure they weren't forgotten. Also, the chocolate truffles were to die for. Needless to say we had an amazing time and left with a couple bottles of their fantastic wines and sparkling rosé. Definitely coming back the next time we're in Santa Barbara!",
 "I had a great time a

In [286]:
combined_review_df = pd.DataFrame()
for i in range(len(combined_review)):
    review = pd.DataFrame([combined_review[i]])
    review["merge"] = review[review.columns[0:]].apply(lambda x: ','.join(x.dropna().astype(str)),axis=1)
    review = review["merge"].to_frame()
    combined_review_df = combined_review_df.append(review)
combined_review_df.columns = ["review_text"]
#combined_review_df = pd.concat([business_CA["business_id"], combined_review_df["review_text"]], axis = 1)
combined_review_df["business_id"] = list(business_CA["business_id"])
combined_review_df["stars"] = list(business_CA["stars"])
#result_df.columns = ["business_id","stars","combined_review"]

In [287]:
combined_review_df

,review_text,business_id,stars
0,"Great beer selection, flat breads are great, g...",-BdYhP-12elmFV7oB1iv4A,5.0
0,My husband and I stopped here once while we we...,-e8RwknT5szoLk9uBZjzcw,4.0
0,My friends and I wanted to come to Llama dog b...,-ujBP1Dw0j1-Ffaz97-LXQ,4.0
0,Food here has always been fresh and delicious....,0CHIbqSkGWBr2KMkIUocEA,3.5
0,So happy we were able to get a reservation thi...,0ac5lGA_0wdKDnib3y59Xg,4.0
...,...,...,...
0,This was so awesome! The night market was a gr...,ySRkAFddd1tCUzRDjbECdg,4.0
0,I love coming here for their drinks and chicke...,ycbmvmBYk4w_ZmdrPL0tBQ,4.0
0,I came last night for the first time in awhile...,ywXa1pDqHPe9tOPx0yJFUg,3.0
0,"Loud,service at bar crazy. They rush around a...",zbrIMldF_O1ZQ0vpUaaa8A,4.0


In [289]:
combined_review_df.to_csv("combined_review.csv",index = False)

### 7. Look for the adjectives around the words (noun) that we chose to analyze

In [290]:
def adjective(noun, text):
    

SyntaxError: unexpected EOF while parsing (644840670.py, line 2)

### 8. Analyze the sentiment score for the nouns.